<a href="https://colab.research.google.com/github/mneedham/data-science-training/blob/master/03_Recommendations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recommendations: Part 1

In this notebook we're going to learn how to make recommendations using Neo4j. Let's import our libraries in case we don't have those from the previous notebooks:

In [ ]:
from py2neo import Graph
import pandas as pd

import matplotlib 
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_colwidth', 100)

Now we need to create a connection to our Neo4j Sandbox. 

<div align="left">
    <img src="images/sandbox-citations2.png" alt="Neo4j 3.4 Sandbox"/>
</div>

We'll need to update the cell below to use the IP Address, Bolt Port, and Password.

In [ ]:
# Change the line of code below to use the IP Address, Bolt Port, and Password of your Sandbox.
# graph = Graph("bolt://<IP Address>:<Bolt Port>", auth=("neo4j", "<Password>")) 
 
graph = Graph("bolt://34.239.255.86:34057", auth=("neo4j", "swords-measurements-routines"))

## Exercise

* Can you find the top 20 suggested collaborators for 'Brian Fitzgerald'?

In [ ]:
query = """
MATCH (author:Author {name: $authorName})<-[:AUTHOR]-(article)-[:AUTHOR]->(coauthor),
      (coauthor)<-[:AUTHOR]-()-[:AUTHOR]->(coc)
WHERE not((coc)<-[:AUTHOR]-()-[:AUTHOR]->(author)) AND coc <> author      
RETURN coc.name AS coauthor, count(*) AS collaborations
ORDER BY collaborations DESC
LIMIT 20
"""

params = {"authorName": "Brian Fitzgerald", "searchTerm": "open source"}
graph.run(query, params).to_data_frame()

* How many of these potential collaborators have collaborated with Brian's collaborators more than 3 times?

In [ ]:
query = """
MATCH (author:Author {name: $authorName})<-[:AUTHOR]-(article)-[:AUTHOR]->(coauthor),
      (coauthor)<-[:AUTHOR]-()-[:AUTHOR]->(coc)
WHERE not((coc)<-[:AUTHOR]-()-[:AUTHOR]->(author)) AND coc <> author      
WITH coc, count(*) AS collaborations
WHERE collaborations > 3
RETURN count(*)
"""
graph.run(query, params).to_data_frame()